In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing, equ2gal
from qubic.lib.Qscene import QubicScene
from qubic.lib.MapMaking.FrequencyMapMaking.Qspectra_component import CMBModel

%matplotlib inline

In [ ]:
nside = 128

# Initialize QUBIC instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
# Build MPI communicator and qubic dict
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubicDict()
d.read_from_file(dictfilename)

center = equ2gal(d['RA_center'], d['DEC_center'])

d['nf_recon'] = 2
d['MultiBand'] = True
d['nside'] = nside

d['npointings'] = 20
d['synthbeam_kmax'] = 1
nf_sub = d['nf_sub']
d['synthbeam_fraction'] = 1

d['noiseless'] = True
d['photon_noise'] = False
d['use_synthbeam_fits_file'] = False

#d['nprocs_instrument'] = 1

In [ ]:
# Build scanning strategy
sampling = get_pointing(d)

# Build scene
scene = QubicScene(d)

In [ ]:
# Number of sub-acquisitions
nsub_max = 32
nsub_list = np.arange(2, nsub_max + 1, 6)

In [ ]:
TOD_list = []
freq_list = []

np.random.seed(d['seed'])

# Build CMB map
cl_cmb = CMBModel(None).give_cl_cmb(r=0, Alens=1)
cmb_map = hp.synfast(cl_cmb, nside, new=True, verbose=False).T

for insub in nsub_list:
    input_maps = np.ones((insub, hp.nside2npix(nside), 3)) * cmb_map[None, ...]
    
    # Build acquisition operator
    Qacq = QubicMultiAcquisitions(d, insub, 2, sampling=sampling)
    
    # Build TOD
    tod = []
    map_list = []
    for i in range(insub):
        P = Qacq.multiinstrument[i].get_projection_operator(sampling, scene)
        F = Qacq.multiinstrument[i].get_filter_operator()
        tod.append(P(F(input_maps[i])))

    TOD_list.append(np.sum(tod, axis=0))
    freq_list.append(np.array(Qacq.allnus))

In [ ]:
np.shape(TOD_list[0])

In [ ]:
# Define a list of colors to use (you can choose your own)
colors = px.colors.qualitative.Plotly

fig = make_subplots(rows=1, cols=3, subplot_titles=("I", "Q", "U"))

for i in range(nsub_list.size):
    # Use the same legend group and color for all three traces of the same i.
    group_name = f"group{i}"
    color = colors[i % len(colors)]
    
    # First subplot: show legend
    fig.add_trace(go.Scatter(
        y=np.mean(TOD_list[i], axis=0)[:, 0],
        name=f"Nsub = {nsub_list[i]}",
        legendgroup=group_name,
        line=dict(color=color, width=1),
        showlegend=True
    ), row=1, col=1)
    
    # Second subplot: hide legend
    fig.add_trace(go.Scatter(
        y=np.mean(TOD_list[i], axis=0)[:, 1],
        legendgroup=group_name,
        line=dict(color=color, width=1),
        showlegend=False
    ), row=1, col=2)
    
    # Third subplot: hide legend
    fig.add_trace(go.Scatter(
        y=np.mean(TOD_list[i], axis=0)[:, 2],
        legendgroup=group_name,
        line=dict(color=color, width=1),
        showlegend=False
    ), row=1, col=3)

fig.update_layout(
    title="TOD - Mean over detector",
    xaxis_title="Pointing",
    yaxis_title="TOD",
)
fig.update_yaxes(tickformat=".1e")  # Example: show scientific notation with 2 decimals

fig.show()


In [ ]:
# Define a list of colors to use (you can choose your own)
colors = px.colors.qualitative.Plotly

fig = make_subplots(rows=1, cols=3, subplot_titles=("I", "Q", "U"))

idet = 200

for i in range(nsub_list.size):
    # Use the same legend group and color for all three traces of the same i.
    group_name = f"group{i}"
    color = colors[i % len(colors)]
    
    # First subplot: show legend
    fig.add_trace(go.Scatter(
        y=TOD_list[i][idet, :, 0],
        name=f"Nsub = {nsub_list[i]}",
        legendgroup=group_name,
        line=dict(color=color, width=1),
        showlegend=True
    ), row=1, col=1)
    
    # Second subplot: hide legend
    fig.add_trace(go.Scatter(
        y=TOD_list[i][idet, :, 1],
        legendgroup=group_name,
        line=dict(color=color, width=1),
        showlegend=False
    ), row=1, col=2)
    
    # Third subplot: hide legend
    fig.add_trace(go.Scatter(
        y=TOD_list[i][idet, :, 2],
        legendgroup=group_name,
        line=dict(color=color, width=1),
        showlegend=False
    ), row=1, col=3)

fig.update_layout(
    title=f"TOD - Detector {idet}",
    xaxis_title="Pointing",
    yaxis_title="TOD",
)
fig.update_yaxes(tickformat=".1e")  # Example: show scientific notation with 2 decimals

fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

for i in range(len(TOD_list)):
    rel_error = np.abs(np.mean(((TOD_list[i] - TOD_list[-1]) / TOD_list[-1])[..., 1]))
    
    ax1.plot(nsub_list[i], rel_error, '.')
    ax2.plot(nsub_list[i], rel_error, '.')

ax1.set_xlabel('Nsub')
ax1.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax1.set_title('Convergence of H (linear scale)')

ax2.set_xlabel('Nsub')
ax2.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax2.set_title('Convergence of H (log scale)')
ax2.set_yscale('log')